#Clasificación de la base de datos CIFAR-10

Esta base de datos tiene 60,000 imágenes a color de 32x32 divididas en 10 clases, teniendo 6000 imágenes por clase. Hay 50,000 imágenes de entrenamiento y 10,000 de prueba. 

Hay 5 conjuntos de entrenamiento y 1 de prueba, para este proyecto se utilizaron todos los conjuntos.

La base de datos se consiguió de la base de datos en línea de Keras

Instala e importa los archivos necesarios de Keras y Tensor Flow

In [0]:
!pip install tensorflow==2.0.0-alpha0

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [0]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Debido a que la base de datos identifica a los datos por número de categoría, aquí se asignan nombres a cada número

In [0]:
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer',
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

Nos dice la forma y el tamaño de nuestras bases de prueba y de entrenamiento

In [0]:
x_train.shape
y_train.shape
len(x_test)
len(x_train)

Hacemos esto para ver una preview de la imagen y saber entre que valores de pixeles se encuentra

In [0]:
plt.figure()
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

Debido a que se decidió usar ReLU como función de activación para el layer, normalizamos los pixeles para que vayan de 0 a 1, por ello se dividirá entre 255

In [0]:
x_train = x_train / 255.0

x_test = x_test / 255.0

Muestra del estado actual de la base de datos

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    aux = np.array(class_names)[y_train[i]]
    plt.xlabel(aux)
plt.show()

Para este ejemplo, se decidió utilizar layers de tipo ReLU debido a su alto grado de efectividad para al análisis de imágenes en líneas de pixeles.

Se usó una layer de output de tipo softmax para poder entregar el resultado con forma probabilística y por su buena sinergia con ReLU

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32,32,3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

Se decidió usar rmsprop

In [0]:
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, epochs=10)

La razón por la que baja su precisión al evaluarlo sobre el conjunto de prueba se debe a que el modelo se comporta de peor manera al ver un conjunto distinto al que usa para entrenar; a esto se le llama fenómeno de sobreajuste

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test)

print('\nTest accuracy:', test_acc)

##Representación gráfica de la efectividad del modelo

In [0]:
predictions = model.predict(x_test)

A continuación se comparará la predicción del tipo de imagen que hará el modelo de la primer imágen del conjunto de prueba contra el tipo de imágen actual

In [0]:
np.argmax(predictions[0])

In [0]:
y_test[0]

Código utilizado para representar cada elemento de la siguiente manera:

Del lado izquierdo la imagen, teniendo en la parte de abajo la categoria adivinada, la probabiliad de escoger dicha categoría y por último la categoría actual. Si se encuentra en azul significa que fue una adivinanza correcta y si es rojo, significa que se equivocó.

Del lado derecho, hay una gráfica en la que se encuentra un gráfico, donde cada barra representa a una de las 10 categorías en orden, siendo la barra y su color la escogida.


In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                np.array(class_names)[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  aux = int(predicted_label)
  aux2 = int(true_label)
  thisplot[aux].set_color('red')
  thisplot[aux2].set_color('blue')

Predicción del primer elemento:

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, y_test, x_test)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  y_test)
plt.show()

Predicción de los primeros 25 elementos

In [0]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, y_test, x_test)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, y_test)
plt.show()